# Assignment 2 / Author Profiling -- Gender and Age

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.classify import apply_features
from nltk.corpus import stopwords
import string
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,ComplementNB
import nltk.metrics
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.preprocessing import LabelEncoder


## Daten einlesen

In [13]:
messages = pd.read_csv('Messages_train.csv', sep=',')

In [14]:
messages.head()

,id,text,person_gender,person_age
0,117808,The particular DKNY bags are among the leading...,male,20s
1,5634,Dental implants are posts made of titanium tha...,female,30s
2,94820,"<p style=""text-align:center;]<a href=""http://e...",female,20s
3,64510,The primary business I joined was the Gospel N...,male,20s
4,89288,Taking QR Codes to Another Level<br />QR Codes...,female,30s


In [31]:
print('Gender :', messages['person_gender'][0] , '  Age class :',messages['person_age'][0] )
print()
print( messages['text'][0][:500] )

Gender : male   Age class : 20s

The particular DKNY bags are among the leading choices of many of the stars each local and also international, politicians, celebrities, tycoons, as well as the designs. These items feature an exceptional sort of style which can be nevertheless way too hard to face up to. As soon as a single units the girl eyes about the most recent design, she'll definitely desire owning that. This kind of brand is really reputable and renowned that over the years, the merchandise are becoming much sought after


In [11]:
df = pd.read_csv("Messages_train.csv", encoding = "latin-1",nrows=500)
stop = stopwords.words('english')
# stemmer_snowball = SnowballStemmer('english')
df['text_without_stopwords'] = df['text'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word.isalpha() and word.lower() not in (stop)]))
df['text_without_stopwords'].str.replace('[{}]'.format(string.punctuation), '')
df['text_without_stopwords'].dropna(inplace=True)
df['gender_age'] = df[['person_gender', 'person_age']].apply(lambda x: ''.join(x), axis=1)


X_train = df.loc[:399, 'text_without_stopwords'].values
y_train_gender = df.loc[:399, 'person_gender'].values

y_train_age = df.loc[:399, 'person_age'].values
X_test = df.loc[400:, 'text_without_stopwords'].values

y_test_gender = df.loc[400:, 'person_gender'].values
y_test_age = df.loc[400:, 'person_age'].values


vectorizer = TfidfVectorizer(analyzer='word', 
                             max_features=100, 
                             sublinear_tf=True)

train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

# classifier with naive bayes
#from sklearn.naive_bayes import MultinomialNB
clf_gender= MultinomialNB().fit(train_vectors, y_train_gender)
clf_age = MultinomialNB().fit(train_vectors, y_train_age)
           
#clf_gender= ComplementNB().fit(train_vectors, y_train_gender)
#clf_age = ComplementNB().fit(train_vectors, y_train_age)


predicted_gender = clf_gender.predict(test_vectors)
predicted_age=clf_age.predict(test_vectors)
print("Gender accuracy",accuracy_score(y_test_gender,predicted_gender))
print("Age accuracy",accuracy_score(y_test_age,predicted_age))


# classifier with SVM
#SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
#clf_gender_svm = SVM.fit(train_vectors, y_train_gender)
#clf_age_svm = SVM.fit(train_vectors, y_train_age)

#predicted_gender_svm = SVM.predict(test_vectors)
#predicted_age_svm = SVM.predict(test_vectors)
#print()
#print('with SVM:')
#print("Gender accuracy",accuracy_score(y_test_gender,predicted_gender_svm))
# print("Age accuracy",accuracy_score(y_test_age,predicted_age_svm))


# creating a confusion matrix for Gender
results = confusion_matrix(y_test_gender, predicted_gender) 
print('\nConfusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test_gender, predicted_gender))
print('Report: ')
print(classification_report(y_test_gender, predicted_gender))

print('Mean:')
print(np.mean(predicted_gender == y_test_gender))

# creating a confusion matrix for Age
results = confusion_matrix(y_test_age, predicted_age) 
print('\nConfusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test_age, predicted_age))
print('Report: ')
print(classification_report(y_test_age, predicted_age))
print('Mean:')
print(np.mean(predicted_age == y_test_age))




(400, 100) (100, 100)
Gender accuracy 0.59
Age accuracy 0.57

Confusion Matrix :
[[10 34]
 [ 7 49]]
Accuracy Score : 0.59
Report: 
              precision    recall  f1-score   support

      female       0.59      0.23      0.33        44
        male       0.59      0.88      0.71        56

   micro avg       0.59      0.59      0.59       100
   macro avg       0.59      0.55      0.52       100
weighted avg       0.59      0.59      0.54       100

Mean:
0.59

Confusion Matrix :
[[ 0  0 15]
 [ 0  0 28]
 [ 0  0 57]]
Accuracy Score : 0.57
Report: 
              precision    recall  f1-score   support

         10s       0.00      0.00      0.00        15
         20s       0.00      0.00      0.00        28
         30s       0.57      1.00      0.73        57

   micro avg       0.57      0.57      0.57       100
   macro avg       0.19      0.33      0.24       100
weighted avg       0.32      0.57      0.41       100

Mean:
0.57


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
X_train = df.loc[:399, 'text_without_stopwords'].values
y_train = df.loc[:399, 'gender_age'].values
X_test = df.loc[400:, 'text_without_stopwords'].values
y_test = df.loc[400:, 'gender_age'].values

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

from sklearn.naive_bayes import MultinomialNB
clf= MultinomialNB().fit(train_vectors, y_train)

predicted = clf.predict(test_vectors)

print("Combined accuracy of Gender and Age",accuracy_score(y_test,predicted))


# creating a confusion matrix for Gender
results = confusion_matrix(y_test, predicted) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, predicted))
print('Report : ')
print(classification_report(y_test, predicted))



(400, 11510) (100, 11510)
Combined accuracy of Gender and Age 0.31
Confusion Matrix :
[[ 0  0  4  0  0  1]
 [ 0  0  4  0  0  3]
 [ 0  0 25  0  0  7]
 [ 0  0  9  0  0  1]
 [ 0  0 13  0  0  8]
 [ 0  0 19  0  0  6]]
Accuracy Score : 0.31
Report : 
              precision    recall  f1-score   support

   female10s       0.00      0.00      0.00         5
   female20s       0.00      0.00      0.00         7
   female30s       0.34      0.78      0.47        32
     male10s       0.00      0.00      0.00        10
     male20s       0.00      0.00      0.00        21
     male30s       0.23      0.24      0.24        25

   micro avg       0.31      0.31      0.31       100
   macro avg       0.09      0.17      0.12       100
weighted avg       0.17      0.31      0.21       100



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Features

In [22]:
def find_features(top_words, text):
    feature = {}
    for word in top_words:
        feature[word] = word in text.lower()
    return feature

bag_of_words = []
for txt in df['text_without_stopwords']:
    for words in txt.split():
        bag_of_words.append(words)
        
# get top 4000 words which will act as our features of each sentence
bag_of_words = nltk.FreqDist(bag_of_words)
top_words = []
for word in bag_of_words.most_common(10000):
    top_words.append(word[0])
top_words[:10]
text_gender = []
c=0
all_genders=df['person_gender']
for txt in df['text_without_stopwords']:
    all_gender=all_genders[c]   
    text_gender.append((txt, all_gender))
    c+=1
    
# creating the feature set, training set and the testing set
feature_set = [(find_features(top_words, text), gender) for (text, gender) in text_gender]
training_set = feature_set[:int(len(feature_set)*4/5)]
testing_set = feature_set[int(len(feature_set)*4/5):]

print("Length of feature set", len(feature_set))
print("Length of training set", len(training_set))
print("Length of testing set", len(testing_set))



Length of feature set 500
Length of training set 400
Length of testing set 100


In [23]:
# creating a classifier using both Gender and Age groups 
NB_classifier = nltk.NaiveBayesClassifier.train(training_set)
accuracy = nltk.classify.accuracy(NB_classifier, testing_set)*100
print("Naive Bayes Classifier accuracy =", accuracy)
NB_classifier.show_most_informative_features(40)

Naive Bayes Classifier accuracy = 59.0
Most Informative Features
                       n = False            male : female =      7.0 : 1.0
             nonetheless = True           female : male   =      6.9 : 1.0
                 burning = True           female : male   =      6.8 : 1.0
                repeated = True           female : male   =      6.8 : 1.0
             inexpensive = True           female : male   =      6.8 : 1.0
                       r = False            male : female =      6.4 : 1.0
                    burn = True           female : male   =      6.0 : 1.0
                 driving = True           female : male   =      6.0 : 1.0
                 chances = True           female : male   =      6.0 : 1.0
         recommendations = True           female : male   =      6.0 : 1.0
                  repeat = True           female : male   =      5.5 : 1.0
                 monitor = True           female : male   =      5.2 : 1.0
                 airport = True    